In [1]:
import json, csv, copy

In [2]:
MAX_RULES = 4
ROWS_TO_PROCESS = 35000

In [18]:
#extract regulations from a row
def get_regulations(row,number=None):
    suffix = "_"+str(number) if number else ""
     
    regulation = {}
   
    rule = {}
    
    if len(row.get('zone'+suffix)):
        rule['activity'] = row.get('zone'+suffix)
    
    if row.get('reason'+suffix):
        rule['reason'] = row.get('reason'+suffix)
    
    if row.get('timeLimit'+suffix):
        rule['maxStay'] = int(row.get('time_limit'+suffix))
        
    if row.get('payment'+suffix):
        rule['payment'] = bool(row.get('payment'+suffix))
    
    if len(rule):
        regulation['rule'] = rule


            
    userClass = {}
    
    if row.get('classes'+suffix):
        userClass['classes'] = [row.get('classes'+suffix)]
    
    if len(userClass):
        regulation['userClass'] = userClass
        
    timeSpans = {}
     
    #Start with the timing:
    if row.get('days_of_week.days'+suffix):
        timeSpans['daysOfWeek'] = {
            'days' : [x.strip() for x in row.get('days_of_week.days'+suffix).split(",")]
        }
        
    if row.get('time_of_day.from'+suffix):
        timeSpans['timesOfDay'] = [{
            'from':row.get('time_of_day.from'+suffix),
            'until'  :row.get('time_of_day.to'+suffix) #Could add defaults here if necessary
        }]
    
    if len(timeSpans):
        regulation['timeSpans'] = [timeSpans]
    else:
        regulation['timeSpans'] = []
        
    # Now check if there are other days or other times of day:
    for suffix2 in ['b','c','d','e','f','g','h','i']:
        new_timeSpans = {}
            
        if row.get('days_of_week.days'+suffix+suffix2):
            new_timeSpans['daysOfWeek'] = {
                'days': [x.strip() for x in row.get('days_of_week.days'+suffix+suffix2).split(",")]
            }


        if row.get('time_of_day.from'+suffix+suffix2):
            if regulation['timeSpans'][0]['daysOfWeek'] == new_timeSpans['daysOfWeek']:
                regulation['timeSpans'][0]['timesOfDay'].append({
                    'from': row.get('time_of_day.from' + suffix + suffix2),
                    'until': row.get('time_of_day.to' + suffix + suffix2) # Could add defaults here if necessary
                })
                
                del new_timeSpans['daysOfWeek']
            else:
                new_timeSpans['timesOfDay'] = [{
                    'from':row.get('time_of_day.from'+suffix+suffix2),
                    'until'  :row.get('time_of_day.to'+suffix+suffix2) #Could add defaults here if necessary
                }]

        if new_timeSpans:
            regulation['timeSpans'].append(new_timeSpans)

    for ts in regulation['timeSpans']:
        if 'daysOfWeek' in ts:
            if len(ts['daysOfWeek']['days']) == 7:
                del ts['daysOfWeek']
            
    payment = {}
    
    if row.get('payment_min' + suffix):
        payment = {
            'rates': [
                {
                    'fees': list(map(float, filter(lambda x: x is not "", [row.get('payment_min' + suffix), row.get('payment_max' + suffix)]))),
                    'durations': list(map(int, filter(lambda x: x is not "", [row.get('payment_min_interval' + suffix), row.get('payment_max_interval' + suffix)])))
                }
            ]
        }
        
        if len(payment['rates'][0]['fees']) == 0:
            del payment['rates'][0]['fees']

        if len(payment['rates'][0]['durations']) == 0:
            del payment['rates'][0]['durations']

    if row.get('method'+suffix):
        payment['method'] = row.get('method'+suffix)
        
    if row.get('payment_form'+suffix):
        payment['paymentForm'] = row.get('payment_form'+suffix)
        
    if len(payment):
        regulation['payment'] = payment
        
    if row.get('priority'+suffix):
        regulation['priority'] = int(row.get('priority'+suffix))
    
    return [regulation]

In [19]:
#Runs everything
import csv
import json

output_rows = []
with open('prepped_data.csv','r') as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        line_count +=1;
        if line_count <= ROWS_TO_PROCESS:
            try:
                #object level
                obj = {
                    'location': {
                        'coordinates': json.loads(row['coordinates']),
                        'shstRefId':row['ShStRefID'],
                        'shstLocationSt':float(row['location_start']),
                        'shstLocationEnd':float(row['location_end']),
                        'sideOfStreet':row['sideOfStreet'],
                        'objectId': row['SpaceID'],
                        'derivedFrom': row['derived_from'].split(","),
                        'marker': row['marker'],
                        'streetName': row['streetName']
                    }
                }

                obj['regulations'] = get_regulations(row,"")

                for i in range(2, MAX_RULES + 1):
                    if row.get("zone_"+str(i)):
                        obj['regulations'].extend(get_regulations(row,i))

                if len(obj['regulations']):
                    output_rows.append(copy.deepcopy(obj))
            except ValueError as e:
                print(e)
                print(row)
                # raise


with open('output.json','w') as outFile:
    json.dump(output_rows,outFile,indent=2)
    


could not convert string to float: 
OrderedDict([('Y', '34.187893'), ('X', '-118.44857'), ('SpaceID', 'VN1009'), ('coordinates', 'null'), ('ShStRefID', 'e4e62cd576344846819e3bd03afde73c'), ('ShStGeomID', '6e466a7a5128e1de547024d94645c68a'), ('feature_id', '25735'), ('marker', 'sign'), ('streetName', '6450 VAN NUYS BL'), ('derived_from', 'VN1009'), ('location_start', '  '), ('location_end', '  '), ('sideOfStreet', 'right'), ('priority', ''), ('zone', ''), ('reason', ''), ('days_of_week.days', ''), ('time_of_day.from', ''), ('time_of_day.to', ''), ('priority_2', '4'), ('zone_2', 'loading'), ('reason_2', 'commercial loading'), ('payment_2', ''), ('class_2', ''), ('days_of_week.days_2', 'Mo, Tu, We, Th, Fr'), ('time_of_day.from_2', '8:00'), ('time_of_day.to_2', '14:00'), ('days_of_week.days_2b', ''), ('time_of_day.from_2b', ''), ('time_of_day.to_2b', ''), ('days_of_week.days_2c', ''), ('time_of_day.from_2c', ''), ('time_of_day.to_2c', ''), ('days_of_week.days_2d', ''), ('time_of_day.from_2

could not convert string to float: 
OrderedDict([('Y', '34.102493'), ('X', '-118.309129'), ('SpaceID', 'HW357'), ('coordinates', 'null'), ('ShStRefID', 'def5b3f5fb8b657a95abcd0b262e38e9'), ('ShStGeomID', 'c6c5871ed3320e203fe3a0e80d0f23f4'), ('feature_id', '11618'), ('marker', 'sign'), ('streetName', '1700 WESTERN AV'), ('derived_from', 'HW357'), ('location_start', '  '), ('location_end', '  '), ('sideOfStreet', 'left'), ('priority', ''), ('zone', ''), ('reason', ''), ('days_of_week.days', ''), ('time_of_day.from', ''), ('time_of_day.to', ''), ('priority_2', '4'), ('zone_2', 'parking'), ('reason_2', ''), ('payment_2', 'TRUE'), ('class_2', ''), ('days_of_week.days_2', 'Mo, Tu, We, Th, Fr, Sa'), ('time_of_day.from_2', '8:00'), ('time_of_day.to_2', '20:00'), ('days_of_week.days_2b', ''), ('time_of_day.from_2b', ''), ('time_of_day.to_2b', ''), ('days_of_week.days_2c', ''), ('time_of_day.from_2c', ''), ('time_of_day.to_2c', ''), ('days_of_week.days_2d', ''), ('time_of_day.from_2d', ''), ('ti

could not convert string to float: 
OrderedDict([('Y', '34.205631'), ('X', '-118.53592'), ('SpaceID', 'R334'), ('coordinates', 'null'), ('ShStRefID', 'caf816e7c1a41791cf8517eed09c6297'), ('ShStGeomID', 'e99c59f86ed01ef907e3f37b38cbfc21'), ('feature_id', '19043'), ('marker', 'sign'), ('streetName', '7400 RESEDA BL'), ('derived_from', 'R334'), ('location_start', '  '), ('location_end', '  '), ('sideOfStreet', 'left'), ('priority', ''), ('zone', ''), ('reason', ''), ('days_of_week.days', ''), ('time_of_day.from', ''), ('time_of_day.to', ''), ('priority_2', '4'), ('zone_2', 'parking'), ('reason_2', ''), ('payment_2', 'TRUE'), ('class_2', ''), ('days_of_week.days_2', 'Mo, Tu, We, Th, Fr, Sa'), ('time_of_day.from_2', '8:00'), ('time_of_day.to_2', '20:00'), ('days_of_week.days_2b', ''), ('time_of_day.from_2b', ''), ('time_of_day.to_2b', ''), ('days_of_week.days_2c', ''), ('time_of_day.from_2c', ''), ('time_of_day.to_2c', ''), ('days_of_week.days_2d', ''), ('time_of_day.from_2d', ''), ('time_o

could not convert string to float: 
OrderedDict([('Y', '34.152388'), ('X', '-118.448558'), ('SpaceID', 'SO578'), ('coordinates', 'null'), ('ShStRefID', 'fbe4dfe89bc6913a2495a6b00539e252'), ('ShStGeomID', '5de07fae561dbd81bda15541b48a3054'), ('feature_id', '22069'), ('marker', 'sign'), ('streetName', '4400 VAN NUYS BL'), ('derived_from', 'SO578'), ('location_start', '  '), ('location_end', '  '), ('sideOfStreet', 'right'), ('priority', ''), ('zone', ''), ('reason', ''), ('days_of_week.days', ''), ('time_of_day.from', ''), ('time_of_day.to', ''), ('priority_2', '4'), ('zone_2', 'parking'), ('reason_2', ''), ('payment_2', 'TRUE'), ('class_2', ''), ('days_of_week.days_2', 'Mo, Tu, We, Th, Fr, Sa'), ('time_of_day.from_2', '8:00'), ('time_of_day.to_2', '20:00'), ('days_of_week.days_2b', ''), ('time_of_day.from_2b', ''), ('time_of_day.to_2b', ''), ('days_of_week.days_2c', ''), ('time_of_day.from_2c', ''), ('time_of_day.to_2c', ''), ('days_of_week.days_2d', ''), ('time_of_day.from_2d', ''), ('

could not convert string to float: 
OrderedDict([('Y', '34.192615'), ('X', '-118.448573'), ('SpaceID', 'VN1148'), ('coordinates', 'null'), ('ShStRefID', 'b81de6c93c2fbb4960af59c20fc55578'), ('ShStGeomID', '9b36d5b71dc3948a2cb40ead88e6fb4f'), ('feature_id', '25856'), ('marker', 'sign'), ('streetName', '6600 VAN NUYS BL'), ('derived_from', 'VN1148'), ('location_start', '  '), ('location_end', '  '), ('sideOfStreet', 'right'), ('priority', ''), ('zone', ''), ('reason', ''), ('days_of_week.days', ''), ('time_of_day.from', ''), ('time_of_day.to', ''), ('priority_2', '4'), ('zone_2', 'parking'), ('reason_2', ''), ('payment_2', 'TRUE'), ('class_2', ''), ('days_of_week.days_2', 'Mo, Tu, We, Th, Fr, Sa'), ('time_of_day.from_2', '8:00'), ('time_of_day.to_2', '18:00'), ('days_of_week.days_2b', ''), ('time_of_day.from_2b', ''), ('time_of_day.to_2b', ''), ('days_of_week.days_2c', ''), ('time_of_day.from_2c', ''), ('time_of_day.to_2c', ''), ('days_of_week.days_2d', ''), ('time_of_day.from_2d', ''), 

could not convert string to float: 
OrderedDict([('Y', '34.16893'), ('X', '-118.610803'), ('SpaceID', 'WH299'), ('coordinates', 'null'), ('ShStRefID', '4f96f6a4a87c86a80e5a19572a663c47'), ('ShStGeomID', 'e91a7a707a1a6e283acf972c3023a6fc'), ('feature_id', '30642'), ('marker', 'sign'), ('streetName', '22201 VENTURA BL'), ('derived_from', 'WH299'), ('location_start', '  '), ('location_end', '  '), ('sideOfStreet', 'right'), ('priority', ''), ('zone', ''), ('reason', ''), ('days_of_week.days', ''), ('time_of_day.from', ''), ('time_of_day.to', ''), ('priority_2', '4'), ('zone_2', 'parking'), ('reason_2', ''), ('payment_2', 'TRUE'), ('class_2', ''), ('days_of_week.days_2', 'Mo, Tu, We, Th, Fr, Sa'), ('time_of_day.from_2', '8:00'), ('time_of_day.to_2', '20:00'), ('days_of_week.days_2b', ''), ('time_of_day.from_2b', ''), ('time_of_day.to_2b', ''), ('days_of_week.days_2c', ''), ('time_of_day.from_2c', ''), ('time_of_day.to_2c', ''), ('days_of_week.days_2d', ''), ('time_of_day.from_2d', ''), ('t

could not convert string to float: 
OrderedDict([('Y', '34.076179'), ('X', '-118.295206'), ('SpaceID', 'WW2597'), ('coordinates', 'null'), ('ShStRefID', '3d4f7394219f5cb52081d381468a3434'), ('ShStGeomID', '53be8f65a7b2f82603fe71dc04d2bd29'), ('feature_id', '34578'), ('marker', 'sign'), ('streetName', '3900 BEVERLY BL'), ('derived_from', 'WW2597'), ('location_start', '  '), ('location_end', '  '), ('sideOfStreet', 'left'), ('priority', ''), ('zone', ''), ('reason', ''), ('days_of_week.days', ''), ('time_of_day.from', ''), ('time_of_day.to', ''), ('priority_2', '4'), ('zone_2', 'parking'), ('reason_2', ''), ('payment_2', 'TRUE'), ('class_2', ''), ('days_of_week.days_2', 'Mo, Tu, We, Th, Fr, Sa'), ('time_of_day.from_2', '8:00'), ('time_of_day.to_2', '18:00'), ('days_of_week.days_2b', ''), ('time_of_day.from_2b', ''), ('time_of_day.to_2b', ''), ('days_of_week.days_2c', ''), ('time_of_day.from_2c', ''), ('time_of_day.to_2c', ''), ('days_of_week.days_2d', ''), ('time_of_day.from_2d', ''), ('

could not convert string to float: 
OrderedDict([('Y', '34.188019'), ('X', '-118.44857'), ('SpaceID', 'VN1011'), ('coordinates', 'null'), ('ShStRefID', 'e4e62cd576344846819e3bd03afde73c'), ('ShStGeomID', '6e466a7a5128e1de547024d94645c68a'), ('feature_id', '25737'), ('marker', 'sign'), ('streetName', '6450 VAN NUYS BL'), ('derived_from', 'VN1011'), ('location_start', '  '), ('location_end', '  '), ('sideOfStreet', 'right'), ('priority', ''), ('zone', ''), ('reason', ''), ('days_of_week.days', ''), ('time_of_day.from', ''), ('time_of_day.to', ''), ('priority_2', '4'), ('zone_2', 'parking'), ('reason_2', ''), ('payment_2', 'TRUE'), ('class_2', ''), ('days_of_week.days_2', 'Mo, Tu, We, Th, Fr, Sa'), ('time_of_day.from_2', '8:00'), ('time_of_day.to_2', '20:00'), ('days_of_week.days_2b', ''), ('time_of_day.from_2b', ''), ('time_of_day.to_2b', ''), ('days_of_week.days_2c', ''), ('time_of_day.from_2c', ''), ('time_of_day.to_2c', ''), ('days_of_week.days_2d', ''), ('time_of_day.from_2d', ''), (

could not convert string to float: 
OrderedDict([('Y', '34.097316'), ('X', '-118.326535'), ('SpaceID', 'SV446'), ('coordinates', 'null'), ('ShStRefID', '5bf689b0da84413974c1af69b99df801'), ('ShStGeomID', '27a4fcf202a969fc1833ae22c20e673d'), ('feature_id', '23550'), ('marker', 'sign'), ('streetName', '1300 VINE ST'), ('derived_from', 'SV446'), ('location_start', '  '), ('location_end', '  '), ('sideOfStreet', 'right'), ('priority', ''), ('zone', ''), ('reason', ''), ('days_of_week.days', ''), ('time_of_day.from', ''), ('time_of_day.to', ''), ('priority_2', '4'), ('zone_2', 'parking'), ('reason_2', ''), ('payment_2', 'TRUE'), ('class_2', ''), ('days_of_week.days_2', 'Mo, Tu, We, Th, Fr, Sa'), ('time_of_day.from_2', '8:00'), ('time_of_day.to_2', '20:00'), ('days_of_week.days_2b', 'Su'), ('time_of_day.from_2b', '11:00'), ('time_of_day.to_2b', '20:00'), ('days_of_week.days_2c', ''), ('time_of_day.from_2c', ''), ('time_of_day.to_2c', ''), ('days_of_week.days_2d', ''), ('time_of_day.from_2d',

KeyError: 'daysOfWeek'

In [20]:
output_rows_geojson = []

for item in output_rows:
    coordinates = item['location'].pop('coordinates')
    output_rows_geojson.append({
        'type':"Feature",
        'geometry':{
            'type':"LineString",
            'coordinates': coordinates
        },
        'properties': item
        
    })


In [21]:
with open('output.geojson','w') as outFile:
    json.dump({'type':'FeatureCollection','features':output_rows_geojson},outFile,indent=2)